In [1]:
import setup
setup.init_django()

In [2]:
import helpers.clients as helper_clients

In [29]:
company_name = "Google"
company_ticker = "GOOG"
multiplier = 1
from_date = "2024-01-09"
to_date = "2024-04-09"

In [30]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

In [31]:
len(dataset)

5000

In [32]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 140.41,
 'close_price': 140.41,
 'high_price': 140.41,
 'low_price': 140.41,
 'number_of_trades': 3,
 'volume': 134,
 'volume_weighted_average': 140.3984,
 'time': datetime.datetime(2024, 1, 9, 9, 2, tzinfo=<UTC>)}

In [33]:
from market.models import Company, StockQuote

In [34]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [35]:
company_obj

<Company: Company object (2)>

In [ ]:
# StockQuote.objects.bulk_create(company=company_obj, **new_stock_data)

In [9]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )

In [12]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>, <StockQuote: StockQuote object (4)>, <StockQuote: StockQuote object (5)>, <StockQuote: StockQuote object (6)>, <StockQuote: StockQuote object (7)>, <StockQuote: StockQuote object (8)>, <StockQuote: StockQuote object (9)>, <StockQuote: StockQuote object (10)>, <StockQuote: StockQuote object (11)>, <StockQuote: StockQuote object (12)>, <StockQuote: StockQuote object (13)>, <StockQuote: StockQuote object (14)>, <StockQuote: StockQuote object (15)>, <StockQuote: StockQuote object (16)>, <StockQuote: StockQuote object (17)>, <StockQuote: StockQuote object (18)>, <StockQuote: StockQuote object (19)>, <StockQuote: StockQuote object (20)>, <StockQuote: StockQuote object (21)>, <StockQuote: StockQuote object (22)>, '...(remaining elements truncated)...']>

In [36]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [37]:
StockQuote.objects.all().count()

10000